In [1]:
import numpy as np
import gc
import cv2
import open3d as o3d
import copy
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
# import sys
# sys.path.append("../static_reconstruction_method/")
# from utils import *

In [8]:
# def from_polar_np(velo):
#     angles = np.linspace(0, np.pi * 2, velo.shape[-1])
#     dist, z = velo[:, 0], velo[:, 1]
#     x = np.cos(angles) * dist
#     y = np.sin(angles) * dist
#     out = np.stack([x,y,z], axis=1)
#     return out.astype('float32')

def draw_pcd(pcd, where='opn_nb'):
    if where is 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where is 'opn_view':
        o3d.visualization.draw_geometries([pcd], width=1280, height=800)
    elif where is 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where is 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()
        
def create_pcd(img):
    some_pcd = o3d.geometry.PointCloud()
    if img.shape[-1] == 3:
        some_pcd.points = o3d.utility.Vector3dVector(img.reshape((-1,3)))
    elif img.shape[-1] == 2:
#         print("on")
        some_arr = from_polar_np(np.expand_dims(polar_img.transpose(2,0,1), axis=0))[0]
        some_arr = some_arr.reshape((3,-1)).T
        some_pcd.points = o3d.utility.Vector3dVector(some_arr)
    elif img.shape[0] == 3:
        some_pcd.points = o3d.utility.Vector3dVector(img.reshape((3,-1)).T)
    else:
        assert False
#     print(some_pcd)
    return some_pcd

def preprocess(dataset, lidar_range):
    # remove outliers 
    min_a, max_a = np.percentile(dataset[:, :, :, [0]], 1), np.percentile(dataset[:, :, :, [0]], 99)
    min_b, max_b = np.percentile(dataset[:, :, :, [1]], 1), np.percentile(dataset[:, :, :, [1]], 99)
    min_c, max_c = np.percentile(dataset[:, :, :, [2]], 1), np.percentile(dataset[:, :, :, [2]], 99)
#     min_a, max_a = -58.183  ,  65.37635
#     min_b, max_b = -60.554525,  62.52805
#     min_c, max_c = -8.3525,  7.695
    print(dataset.shape)
    print("Min_x:{} Max_x:{} | Min_y:{} Max_y:{} | Min_z:{} Max_z:{}".format(min_a, max_a, min_b, max_b, min_c, max_c))
    print("Finding crop masks")
    mask = np.maximum(dataset[:, :, :, 0] < min_a, dataset[:, :, :, 0] > max_a)
    mask = np.maximum(mask, np.maximum(dataset[:, :, :, 1] < min_b, dataset[:, :, :, 1] > max_b))
    mask = np.maximum(mask, np.maximum(dataset[:, :, :, 2] < min_c, dataset[:, :, :, 2] > max_c))
    dist = dataset[:, :, :, 0] ** 2 + dataset[:, :, :, 1] ** 2
    mask = np.maximum(mask, dist < 7)
    print("Masking")
    dataset = dataset * (1 - np.expand_dims(mask, -1))

    dataset /= lidar_range   # Max LIDAR value
    dataset = to_polar_np(dataset).transpose(0, 3, 1, 2)
    previous = (dataset[:, 0] == 0).sum()

    remove = []
    print("Remove zeros")
    for i in trange(dataset.shape[0]):
        try:
            pp = remove_zeros(dataset[i]).squeeze(0)
            dataset[i] = pp
        except Exception as e:
            print("Exception in zeros functions : {}".format(e))
            remove += [i]

    for i in remove:
        dataset = np.concatenate([dataset[:i-1], dataset[i+1:]], axis=0)
    
    return dataset

In [2]:
# NPY_PATH = "/home/saby/Projects/ati/ati_motors/adversarial_based/training_data/ati/preprocess-[2-16-1024]/s1.npy"

# arr = np.load(NPY_PATH)
# arr.shape

In [3]:
# polar_img = arr[0]
# print(polar_img.shape)

# img = from_polar_np(np.expand_dims(polar_img, axis=0))[0]
# img = img.transpose(1,2,0)
# print(img.shape)

In [4]:
def to_polar_np(velo):
    if len(velo.shape) == 4:
        velo = velo.transpose(1, 2, 3, 0)

    if velo.shape[2] > 4:
        assert velo.shape[0] <= 4
        velo = velo.transpose(1, 2, 0, 3)
        switch=True
    else:
        switch=False
    
    # assumes r x n/r x (3,4) velo
    dist = np.sqrt(velo[:, :, 0] ** 2 + velo[:, :, 1] ** 2)
    # theta = np.arctan2(velo[:, 1], velo[:, 0])
    out = np.stack([dist, velo[:, :, 2]], axis=2)
    
    if switch:
        out = out.transpose(2, 0, 1, 3)

    if len(velo.shape) == 4: 
        out = out.transpose(3, 0, 1, 2)
    
    return out

def from_polar_np(velo):
    angles = np.linspace(0, np.pi * 2, velo.shape[-1])
    dist, z = velo[:, 0], velo[:, 1]
    x = np.cos(angles) * dist
    y = np.sin(angles) * dist
    out = np.stack([x,y,z], axis=1)
    return out.astype('float32')

def remove_zeros(pc):
    xx = torch.cuda.FloatTensor(pc)
    if xx.dim() == 3: 
        xx = xx.unsqueeze(0)

    iters = 0
    pad = 2
    ks = 5
    while (xx[:, 0] == 0).sum() > 0 : 
        if iters  > 100:
            print("Exceeded {} iterations".format(iters))
            raise ValueError()
            ks += 2
            pad += 1
        
        mask = (xx[:, 0] == 0).unsqueeze(1).float()
        out_a = F.max_pool2d(xx[:, 0], ks, padding=pad, stride=1)
#         out_b = -F.max_pool2d(-xx[:, 1], ks, padding=pad, stride=1)
        out_b = F.max_pool2d(xx[:, 1], ks, padding=pad, stride=1)   # Changing this since our z = -z from carla
        #out_b_ = (xx[:, 1]).min(dim=-1, keepdim=True)[0].expand_as(out_b)
        #out_b = torch.cat([out_b_[:, :10], out_b[:, 10:]], dim=1)
        out_b = out_b.expand_as(out_a)
        out = torch.stack([out_a, out_b], dim=1)
        mask = (xx[:, 0] == 0).unsqueeze(1)
        mask = mask.float()
        xx = xx * (1 - mask) + (mask) * out
        iters += 1

    return xx.cpu().data.numpy()

def preprocess_dataset_for_interpolate(dataset):
    print("Preprocessing")
    dist = dataset[:, :, :, 0] ** 2 + dataset[:, :, :, 1] ** 2
    mask = (dist < 7)
    dataset = dataset * (1 - np.expand_dims(mask, -1))

    dataset = to_polar_np(dataset).transpose(0, 3, 1, 2)
    for i in trange(dataset.shape[0]):
        try:
            pp = remove_zeros(dataset[i]).squeeze(0)
            dataset[i] = pp
        except Exception as e:
            print("Exception in zeros functions : {}".format(e))
            remove += [i]
    return dataset

def interpolate_for_not_ground_image(img, scaleby):
    img_z = img[:,:,2]
    img_z_linear = cv2.resize(img_z,None,fx=1,fy=scaleby, interpolation = cv2.INTER_LINEAR)
    img_xy = img[:,:,:2]
    img_xy_nearest = cv2.resize(img_xy,None,fx=1,fy=scaleby, interpolation = cv2.INTER_NEAREST)

    new_img = np.concatenate((img_xy_nearest, np.expand_dims(img_z_linear, axis=-1)), axis=-1)
    return new_img

def interpolate_for_ground_image(img, scaleby):
    img_z = img[:,:,2]
    img_z_nearest = cv2.resize(img_z,None,fx=1,fy=scaleby, interpolation = cv2.INTER_NEAREST)
    img_xy = img[:,:,:2]
    img_xy_linear = cv2.resize(img_xy,None,fx=1,fy=scaleby, interpolation = cv2.INTER_LINEAR)

    new_img = np.concatenate((img_xy_linear, np.expand_dims(img_z_nearest, axis=-1)), axis=-1)
    return new_img

def beam_interpolation(arrayray, scaleby=4, mask_thresh=0.75):  # scaleby is 4 for going from 16 to 64
    # Remove zeros
    dataset = preprocess_dataset_for_interpolate(arrayray)
    dataset = dataset.transpose(0,2,3,1)

    print("Create new dataset")
    new_dataset_list = []
    for polar_img in tqdm(dataset):
        # polar_img = dataset[100,:,:]
        # Convert back to xyz
        img = from_polar_np(np.expand_dims(polar_img.transpose(2,0,1), axis=0))[0].transpose(1,2,0)

        # Get Ground mask
        img_z = img[:,:,2]
        mask_channel = (img_z < mask_thresh).astype(int)
        mask_not_ground = np.expand_dims(mask_channel, axis=-1)
        
        # Interpolate separately for ground and non-ground points
        img_ground  = img * (1-mask_not_ground)
        new_img_ground = interpolate_for_ground_image(img_ground, scaleby)        
        img_not_ground  = img * mask_not_ground
        new_img_not_ground = interpolate_for_not_ground_image(img_not_ground, scaleby)
        new_mask_channel = cv2.resize(mask_not_ground,None,fx=1,fy=scaleby, interpolation = cv2.INTER_NEAREST)
        new_mask_not_ground = np.expand_dims(new_mask_channel, axis=-1)

        # Extract the new image from separate interpolations based on ground mask
        new_img = (new_img_ground * (1-new_mask_not_ground)) + (new_img_not_ground * new_mask_not_ground)
        new_dataset_list.append(new_img)
        gc.collect()

    new_dataset = np.array(new_dataset_list)
    return new_dataset

In [5]:
NPY_PATH = "/home/saby/Projects/ati/data/data/datasets/Real_World/pair_transform/dynamic_out_npy/1.npy"
arr = np.load(NPY_PATH)
print(arr.shape)
new_arr = beam_interpolation(arr, scaleby=4)
print(new_arr.shape)

(687, 16, 1024, 4)
Preprocessing


  1%|          | 4/687 [00:00<00:20, 33.43it/s]

Create new dataset


100%|██████████| 687/687 [00:21<00:00, 32.43it/s]


(687, 64, 1024, 3)


In [ ]:
assert False

In [ ]:
dataset.shape

In [ ]:
dataset = dataset.transpose(0,2,3,1)
dataset.shape

In [ ]:
polar_img = dataset[100,:,:]
polar_img.shape

In [ ]:
# img=polar_img
img = from_polar_np(np.expand_dims(polar_img.transpose(2,0,1), axis=0))[0].transpose(1,2,0)
img.shape

In [10]:
prev_img = arr[0,:,:,:3]
img = new_arr[0]

In [11]:
print(prev_img.shape)
draw_pcd(create_pcd(prev_img))

(16, 1024, 3)


JVisualizer with 1 geometries

In [12]:
print(img.shape)
draw_pcd(create_pcd(img))

(64, 1024, 3)


JVisualizer with 1 geometries

In [ ]:
filtered_pcd, _ = this_pcd.remove_radius_outlier(nb_points = 20, radius = 0.75)
draw_pcd(filtered_pcd)

In [ ]:
img_z = img[:,:,2]
img_z_linear = cv2.resize(img_z,None,fx=1,fy=4, interpolation = cv2.INTER_LINEAR)
print(img_z_linear.shape)

img_xy = img[:,:,:2]
img_xy_nearest = cv2.resize(img_xy,None,fx=1,fy=4, interpolation = cv2.INTER_NEAREST)
print(img_xy_nearest.shape)

new_img = np.concatenate((img_xy_nearest, np.expand_dims(img_z_linear, axis=-1)), axis=-1)
print(new_img.shape)

new_this_pcd = create_pcd(new_img)
draw_pcd(new_this_pcd)

In [ ]:
new_filtered_pcd, _ = new_this_pcd.remove_radius_outlier(nb_points = 20, radius = 1)
draw_pcd(new_filtered_pcd)

In [ ]:
img.shape

In [ ]:
img_z = img[:,:,2]
mask_channel = (img_z < 0.75).astype(int)
mask_not_ground = np.expand_dims(mask_channel, axis=-1)
img_ground  = img * (1-mask_not_ground)
img_not_ground  = img * mask_not_ground

In [ ]:
new_img_ground = interpolate_for_ground_image(img_ground)
new_img_not_ground = interpolate_for_not_ground_image(img_not_ground)
new_mask_channel = cv2.resize(mask_not_ground,None,fx=1,fy=4, interpolation = cv2.INTER_NEAREST)
new_mask_not_ground = np.expand_dims(new_mask_channel, axis=-1)

In [ ]:
new_img = (new_img_ground * (1-new_mask_not_ground)) + (new_img_not_ground * new_mask_not_ground)

In [ ]:
new_this_pcd =  create_pcd(new_img)
new_filtered_pcd, _ = new_this_pcd.remove_radius_outlier(nb_points = 10, radius = 0.75)
draw_pcd(new_filtered_pcd)